**Enable autoreload extension**

In [1]:
%load_ext autoreload
%autoreload 3

**Setup dependencies**

In [2]:
!apt-get update --fix-missing
!apt-get install -y libspatialindex-dev
%pip install --upgrade pip wheel
%pip install eclipse-sumo==1.10.0
%env SUMO_HOME=/usr/local/lib/python3.7/dist-packages/sumo
%pip install git+https://github.com/huawei-noah/SMARTS.git@2b2179b62bf7b6a34bf66712a9d071a081f2a345#egg=smarts[camera-obs]
!rm -rf /content/SMARTS
!git clone https://github.com/huawei-noah/SMARTS /content/SMARTS

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 https://developer.download.nvidia.com/compute/c

**Import Base Modules**

In [3]:
import gym

import smarts
import smarts.core
import smarts.env

**Setup Debug Logging Temporarily**

In [4]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

**Define a new agent for this example**

In [5]:
from smarts.core.agent import Agent
from smarts.core.sensors import Observation

class ChaseViaPointsAgent(Agent):
    def act(self, obs: Observation):
        if (
            len(obs.via_data.near_via_points) < 1
            or obs.ego_vehicle_state.road_id != obs.via_data.near_via_points[0].road_id
        ):
            return (obs.waypoint_paths[0][0].speed_limit, 0)

        nearest = obs.via_data.near_via_points[0]
        if nearest.lane_index == obs.ego_vehicle_state.lane_index:
            return (nearest.required_speed, 0)

        return (
            nearest.required_speed,
            1 if nearest.lane_index > obs.ego_vehicle_state.lane_index else -1,
        )

2022-02-21 01:50:38,409 - shapely.geos - DEBUG - Found GEOS DLL: <CDLL '/usr/local/lib/python3.7/dist-packages/Shapely.libs/libgeos_c-74dec7a7.so.1.14.2', handle 5592032ce300 at 0x7fa7a5363c10>, using it.


**Create an Environment (likely elsewhere)**

In [6]:
from smarts.core.agent import AgentSpec
from smarts.core.agent_interface import AgentInterface, AgentType
from smarts.env.wrappers.single_agent import SingleAgent

agent_spec = AgentSpec(
    interface=AgentInterface.from_type(
        AgentType.LanerWithSpeed,
        max_episode_steps=100,
        rgb=True
    ),
    agent_builder=ChaseViaPointsAgent
)

def entry_point(*args, **kwargs):
  from smarts.env.hiway_env import HiWayEnv
  scenario = "/content/SMARTS/scenarios/figure_eight"
  ## Note: can build the scenario here
  from cli.studio import _build_single_scenario
  _build_single_scenario(clean=True, allow_offset_map=True, scenario=scenario)
  return SingleAgent(
    HiWayEnv(
      agent_specs={"agent-007": agent_spec},
      scenarios=[scenario],
      headless=True,
      sumo_headless=True,
  ))

gym.register(
    "figure_eight-v1",
    entry_point=entry_point
)

**Run an episode**

In [7]:
from smarts.core.utils.episodes import episodes


env = gym.make("figure_eight-v1")

for episode in episodes(n=1):
    agent = agent_spec.build_agent()
    observation = env.reset()
    episode.record_scenario(env.scenario_log)

    done = False
    while not done:
        agent_action = agent.act(observation)
        observation, reward, done, info = env.step(agent_action)
        episode.record_step(observation, reward, done, info)

env.close()
print ('Success')

2022-02-21 01:50:39,631 - trimesh - DEBUG - falling back to MD5 hashing: `pip install xxhash` for 30x faster cache checks


/usr/local/lib/python3.7/dist-packages/html5lib/_trie/_base.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


build-scenario /content/SMARTS/scenarios/figure_eight
2022-02-21 01:50:42,329 - trimesh - INFO - triangulating quad faces
2022-02-21 01:50:42,341 - trimesh - DEBUG - mesh transformed by matrix
2022-02-21 01:50:42,358 - trimesh - DEBUG - generating face normals
╭────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────╮
│            Episode │     Sim T / Wall T │        Total Steps │        Steps / Sec │       Scenario Map │    Scenario Routes │     Mission (Hash) │             Scores │
├────────────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┤
2022-02-21 01:50:42,968 - AgentManager - DEBUG - Tearing down AgentManager
2022-02-21 01:50:42,972 - SumoTrafficSimulation - DEBUG - Tearing down SUMO traffic sim SumoTrafficSim(
  _scenario=None,
  _time_resolution=0.1,
  _headl

RendererException: ignored